In [20]:
import pandas as pd
import numpy as np
import time
from matplotlib import pyplot as plt
import datetime as dt
import talib
from tqdm import tqdm
import colorama as col
from bt import *
from sklearn.preprocessing import *
from sklearn.pipeline import Pipeline

print('Ready')

Ready


In [19]:
%%time
##### Chargement de la base
base = pd.read_csv('Base_Clean/EURUSDm5.csv')

##### On fixe la date en index sous forme de Timestamp
base.set_index(pd.to_datetime(base.Date),drop=True,inplace=True)

###### On drop les colonnes inutiles
base = base.drop(['Date','Total'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
base['WE'] = np.where(((base.index.weekday == 5) | (base.index.weekday == 6)),None,base.index.weekday)
base = base.dropna()
base = base.drop(['WE'],axis=1)


CPU times: user 1.38 s, sys: 116 ms, total: 1.5 s
Wall time: 1.49 s


In [6]:
base.index[0],base.index[-1]

(Timestamp('2010-11-23 00:00:00'), Timestamp('2020-11-23 15:40:00'))

In [7]:
%%time
_year_bottom = pd.to_datetime('2011-01-01 16:00:00')
_year_top = pd.to_datetime('2015-12-31 16:00:00') #2020-11-23 16:00:00

df = base[(base.index>=_year_bottom)&(base.index<=_year_top)]

CPU times: user 19.9 ms, sys: 10.7 ms, total: 30.5 ms
Wall time: 29.3 ms


In [8]:
df['HiWin'] = df.High.rolling(10).max()
df['LoWin'] = df.Low.rolling(10).min()
df['JyssOscUp'] = ((df.HiWin-df.HiWin.shift(2))/df.HiWin.shift(2))*1000
df['JyssOscUpSD'] = df.JyssOscUp.rolling(20).std()
df['JyssOscDwn'] = ((df.LoWin-df.LoWin.shift(2))/df.LoWin.shift(2))*1000
df['JyssOscDwnSD'] = df.JyssOscDwn.rolling(20).std()
df['TriggerBull'] = (df.HiWin - df.HiWin.shift(2)) * 1000
df['TriggerBear'] = (df.LoWin - df.LoWin.shift(2)) * 1000
df['Trigger'] = np.where((df.LoWin - df.LoWin.shift(9) == 0),1,np.where((df.HiWin - df.HiWin.shift(9) == 0 ),1,0))

In [9]:
df['Signal'] = np.where(\
    ((df.Close <= 1.01*df.LoWin)&(df.JyssOscUp <= -4*df.JyssOscUpSD)),1,\
        np.where(\
            ((df.Close >= 0.99*df.HiWin)&(df.JyssOscDwn >= 4*df.JyssOscDwnSD)),-1,0)
)


In [10]:
_nb_bougie_exit = 70
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 0
_verbose = 0
_cash_ini = 200000
_rate = 1
x = 'EUR/USD'
_target = 0.002
_exposure = 3
_size = 50000
_sl = 0.002
TRACKER = bt(df,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

Librairies imported

Début des opérations horodatée à 2020-12-23 20:04:16.979173

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 369102/369102 [00:11<00:00, 31393.29it/s]
 Pour le ticker  EUR/USD 
 
Les gains faramineux s'élèvent à : $  -2981.0 !. En  1899   transactions.
 
Nbre de winners : 1035 
 
Nbre de loosers : 864 
Temps d'excution du BT 0:00:18.310665
 EUR/USD  results 
 Tested Period 2011-01-01 16:00:00  à 2015-12-31 16:00:00 
 Total Number of trades 1899 
Started Cash : 50000
P&L in currency:  -2981.0$ 
P&L in %:  -1.49% 
Average trade duration 58.02
# Winners  1035.0
# Loosers  864.0
Cumulated gains 110448.0
Cumulated losses -113429.0
 PROFIT FACTOR :  0.97 
 Winners Ratio : 54.5 % 
Average Winners 107.13
% Average Winners 0.05
Average Loosers -131.28
% Average Loosers -0.07
Average pnl -1.57
% Average pnl -0.0
Number of opened trades 1899
Number of closed trades 1899
Max Ex

In [11]:
len(TRACKER)

1040

In [12]:
%%time
scaler = MinMaxScaler((0,1))
df_2 = df.copy()
df_2['RSI2'] = talib.RSI(df_2.Close,timeperiod=2)
df_2['MMA200'] = talib.EMA(df_2.Close,timeperiod=200)
upper, middle, lower = talib.BBANDS(df_2.Close, timeperiod=10, nbdevup=2, nbdevdn=2)
df_2['SBVol'] = upper - lower
df_2['MMA55'] = talib.EMA(df_2.Close,timeperiod=55)
df_2['MMA80'] = talib.EMA(df_2.Close,timeperiod=80)
df_2['RSI14'] = talib.RSI(df_2.Close,timeperiod=14)

ar_up,ar_low = talib.AROON(df_2.High,df_2.Low,10)
df_2['AR_HI'] = ar_up
df_2['AR_LO'] = ar_low
df_2['AROON'] = ar_up - ar_low

#df_2 = df_2.drop(['SignalClose'],axis=1)

df_2.rename(columns = {'Signal':'SigO4bt'}, inplace = True)
SIGNAL = []
SIGNAL = [1 if ((df_2['SigO4bt'][i] != 0) & (df_2.index[i] in TRACKER)) else 0 for i in tqdm(range(len(df_2)))]
df_2['Signal'] = SIGNAL

df_2 = df_2.dropna()

df_2 = df_2.sort_index(axis=1)


100%|██████████| 369132/369132 [00:23<00:00, 15752.28it/s]
CPU times: user 23.6 s, sys: 263 ms, total: 23.8 s
Wall time: 23.8 s


In [13]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 368933 entries, 2011-01-03 16:35:00 to 2015-12-31 16:00:00
Data columns (total 34 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   AROON         368933 non-null  float64
 1   AR_HI         368933 non-null  float64
 2   AR_LO         368933 non-null  float64
 3   Close         368933 non-null  float64
 4   CloseAsk      368933 non-null  float64
 5   CloseBid      368933 non-null  float64
 6   HiWin         368933 non-null  float64
 7   HigMax        368933 non-null  float64
 8   High          368933 non-null  float64
 9   HighAsk       368933 non-null  float64
 10  HighBid       368933 non-null  float64
 11  JyssOscDwn    368933 non-null  float64
 12  JyssOscDwnSD  368933 non-null  float64
 13  JyssOscUp     368933 non-null  float64
 14  JyssOscUpSD   368933 non-null  float64
 15  LoWin         368933 non-null  float64
 16  Low           368933 non-null  float64
 17  LowAsk        

In [14]:
%%time
poly = PolynomialFeatures(3)
df_2['_AROON'] = poly.fit_transform(
    X = df_2['AROON'].values.reshape(-1,1),
    y = df_2['Signal']
    )[:,3]

CPU times: user 12.8 ms, sys: 1.99 ms, total: 14.8 ms
Wall time: 14.3 ms


In [15]:
_C = 1.0
# Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive. The penalty is a squared l2 penalty.

_kernel = 'poly'
#{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
#Specifies the kernel type to be used in the algorithm. It must be one of ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ or a callable. If none is given, ‘rbf’ will be used. If a callable is given it is used to pre-compute the kernel matrix from data matrices; that matrix should be an array of shape (n_samples, n_samples).

_degree = 3
#int, default=3
#Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.

_gamma = 'scale'
#{‘scale’, ‘auto’} or float, default=’scale’
#Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.

#if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,
#if ‘auto’, uses 1 / n_features.
#Changed in version 0.22: The default value of gamma changed from ‘auto’ to ‘scale’.
_coef = 0.0
#0float, default=0.0
#Independent term in kernel function. It is only significant in ‘poly’ and ‘sigmoid’.

_shrinking = True
#bool, default=True
#Whether to use the shrinking heuristic. See the User Guide.

_probability = False
#bool, default=False
#Whether to enable probability estimates. This must be enabled prior to calling fit, will slow down that method as it internally uses 5-fold cross-validation, and predict_proba may be inconsistent with predict. Read more in the User Guide.

_tol = 0.003
#float, default=1e-3
#Tolerance for stopping criterion.

_cache_size = 200 
# float, default=200
#Specify the size of the kernel cache (in MB).

_class_weight = None
#dict or ‘balanced’, default=None
#Set the parameter C of class i to class_weight[i]*C for SVC. If not given, all classes are supposed to have weight one. The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))

_verbose = False
#bool, default=False
#Enable verbose output. Note that this setting takes advantage of a per-process runtime setting in libsvm that, if enabled, may not work properly in a multithreaded context.

_max_iter = -1
#int, default=-1
#Hard limit on iterations within solver, or -1 for no limit.

_decision_function_shape = 'ovr'
#{‘ovo’, ‘ovr’}, default=’ovr’
#Whether to return a one-vs-rest (‘ovr’) decision function of shape (n_samples, n_classes) as all other classifiers, or the original one-vs-one (‘ovo’) decision function of libsvm which has shape (n_samples, n_classes * (n_classes - 1) / 2). However, one-vs-one (‘ovo’) is always used as multi-class strategy. The parameter is ignored for binary classification.

#Changed in version 0.19: decision_function_shape is ‘ovr’ by default.
#New in version 0.17: decision_function_shape=’ovr’ is recommended.

#Changed in version 0.17: Deprecated decision_function_shape=’ovo’ and None.
_break_ties = False
#bool, default=False
#If true, decision_function_shape='ovr', and number of classes > 2, predict will break ties according to the confidence values of decision_function; otherwise the first class among the tied classes is returned. Please note that breaking ties comes at a relatively high computational cost compared to a simple predict.

#New in version 0.22.

_random_state = 53
#int or RandomState instance, default=None

In [16]:
%%time
from svm import modelize

x = 'EUR/USD'
_model = 'svm'

modelize(df=df_2[df_2.SigO4bt != 0],x=x,_model=_model,_verb=0,C=_C, kernel=_kernel, degree=_degree, gamma=_gamma, coef0=_coef0, shrinking=_shrinking, probability=_probability, tol=_tol, cache_size=_cache_size, class_weight=_class_weight, verbose=_verbose, max_iter=_max_iter, decision_function_shape=_decision_function_shape, break_ties=_break_ties, random_state=_random_state)

ModuleNotFoundError: No module named 'svm'